In [ ]:
# 코랩에서 구글 드라이브 접근
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models.resnet import BasicBlock
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models, datasets
from PIL import Image
import pandas as pd
import os

In [ ]:
class SteelDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, str(self.data.iloc[idx, 1]), self.data.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.data.iloc[idx, 1] - 1  # Adjust label to start from 0
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# 하이퍼파라미터 설정
batch_size = 32
learning_rate = 0.001
num_classes = 4

# 데이터 로드 및 전처리
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = SteelDataset(csv_path='/content/drive/MyDrive/졸업과제/severstal-steel-defect-detection/aug-train.csv',
                              img_dir='/content/drive/MyDrive/졸업과제/working/aug_train/',
                              transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class ResNetSteelClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetSteelClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetSteelClassifier(num_classes=4).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 61.6MB/s]


In [ ]:
from tqdm import tqdm

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(train_loader, unit="batch") as t:
        for images, labels in t:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            t.set_postfix(loss=running_loss / (t.n + 1))

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

100%|██████████| 125/125 [20:35<00:00,  9.88s/batch, loss=1.21]


Epoch [1/20], Loss: 1.2096


100%|██████████| 125/125 [00:40<00:00,  3.07batch/s, loss=1.08]


Epoch [2/20], Loss: 1.0801


100%|██████████| 125/125 [00:40<00:00,  3.12batch/s, loss=1.03]


...

Epoch [18/20], Loss: 0.7980


100%|██████████| 125/125 [00:39<00:00,  3.16batch/s, loss=0.787]


Epoch [19/20], Loss: 0.7874


100%|██████████| 125/125 [00:41<00:00,  2.98batch/s, loss=0.771]

Epoch [20/20], Loss: 0.7712


In [ ]:
# 학습된 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/졸업과제/모델/resnet_model_epoch_20.pth')

In [ ]:
# 모델 평가
model.eval()
preds = []
targets = []

with torch.no_grad(), tqdm(train_loader, unit="batch") as t:
    for images, labels in t:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        targets.extend(labels.cpu().numpy())
        preds.extend(predicted.cpu().numpy())
        t.set_postfix(accuracy=accuracy_score(targets, preds))

# 평가 지표 계산
accuracy = accuracy_score(targets, preds)
precision = precision_score(targets, preds, average='weighted')
recall = recall_score(targets, preds, average='weighted')
f1 = f1_score(targets, preds, average='weighted')

print()
print("Train")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

100%|██████████| 125/125 [00:34<00:00,  3.67batch/s, accuracy=0.567]


Train
Accuracy: 0.5673
Precision: 0.6002
Recall: 0.5673
F1 Score: 0.5698


In [ ]:
# 테스트 데이터 로드 및 전처리
test_dataset = SteelDataset(csv_path='/content/drive/MyDrive/졸업과제/severstal-steel-defect-detection/aug-test.csv',
                              img_dir='/content/drive/MyDrive/졸업과제/working/aug-test/',
                              transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

# 모델 평가 및 평가 지표 계산 (테스트 데이터)
test_preds = []
test_targets = []

with torch.no_grad():
    with tqdm(test_loader, unit="batch") as t:
        for images, labels in t:
            images = images.cuda()
            labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_preds.extend(predicted.cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

            t.set_postfix(test_accuracy=accuracy_score(test_targets, test_preds))

# 평가 지표 계산 (테스트 데이터)
test_accuracy = accuracy_score(test_targets, test_preds)
test_precision = precision_score(test_targets, test_preds, average='weighted')
test_recall = recall_score(test_targets, test_preds, average='weighted')
test_f1 = f1_score(test_targets, test_preds, average='weighted')

print()
print("Test")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1 Score: {test_f1:.4f}")

100%|██████████| 32/32 [05:41<00:00, 10.68s/batch, test_accuracy=0.549]


Test
Accuracy: 0.5490
Precision: 0.5722
Recall: 0.5490
F1 Score: 0.5487


### 저장된 모델 불러와서 사용

In [ ]:
# 1. 모델 아키텍처 정의 및 초기화
model = ResNetSteelClassifier(num_classes=4)  # 예시로 사용한 모델 아키텍처로 초기화

# 2. 저장한 모델 파라미터 불러오기
saved_model_path = '/content/drive/MyDrive/졸업과제/모델/resnet_model_epoch_10.pth'
model.load_state_dict(torch.load(saved_model_path))
model.to(device)
model.eval()  # 모델을 평가 모드로 설정

# 3. 입력 데이터로 예측 수행
input_image = ...  # 예측하고자 하는 이미지 데이터
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = transform(input_image).unsqueeze(0).to(device)  # 배치 차원 추가 및 디바이스 설정

with torch.no_grad():
    output = model(input_tensor)
    _, predicted_class = torch.max(output, 1)

predicted_class = predicted_class.item()
print(f'Predicted class: {predicted_class}')

### 직접 구현된 코드

In [ ]:
# ResNet 모델 정의
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=num_classes):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        layers = []
        layers.append(block(self.in_planes, planes, stride))
        self.in_planes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# ResNet 모델 인스턴스 생성 및 GPU로 이동
model = ResNet(BasicBlock, [2, 2, 2, 2]).cuda()

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished!")